# Build a Crew to Tailor Job Applications

In this lesson, you will build a multi-agent system.

 If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [8]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [9]:
from crewai import Agent, Task, Crew

In [10]:
import os
os.chdir('/workspaces/codespaces-jupyter')
print("Changed working directory to:", os.getcwd())

Changed working directory to: /workspaces/codespaces-jupyter


**Note**: 
- For today we will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ 

In [11]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [12]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

2026-01-22 02:02:36,915 - 131791170269824 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientStartEvent: module 'chromadb' has no attribute 'get_settings'
2026-01-22 02:02:36,918 - 131791170269824 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'
2026-01-22 02:02:36,919 - 131791170269824 - posthog.py-posthog:59 - ERROR: Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'
Inserting batches in chromadb:   0%|          | 0/1 [00:03<?, ?it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [13]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [14]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [15]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [16]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [17]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [18]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [19]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [20]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [21]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [22]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [29]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/timmasterson/',
    'personal_writeup': """I'm one of the key digital leaders at Paramount Streaming and the Paramount+ platform. I'm the Principal Architect for Web and Smart TV applications. I have a long history of getting big things done when it counts. I was the architect for Paramount+ Web during the global rollout and our global code merge. I was also part of hosting Super Bowl 58 on Paramount+. Before that I led the technical integration of Tele-Health and AI features into Sydney Health for Anthem (now Elevance) and a few other projects with global impact."""
    
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [32]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/timmasterson/) URLs, and personal write-up (I'm one of the key digital leaders at Paramount Streaming and the Paramount+ platform. I'm the Principal Architect for Web and Smart TV applications. I have a long history of getting big things done when it counts. I was the architect for Paramount+ Web during the global rollout and our global code merge. I was also part of hosting Super Bowl 58 on Paramo

- Dislplay the generated `tailored_resume.md` file.

In [33]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Tim Masterson

## Contact Information
- Email: tim.masterson@example.com
- Phone: (555) 123-4567
- LinkedIn: (https://www.linkedin.com/in/tim-masterson-a202574/)

## Summary
Principal Architect with extensive experience in streaming platforms, web applications, and smart TV development. Proven track record of delivering large-scale projects with global impact. Seeking a challenging role in AI Fund to utilize technical expertise and leadership skills.

## Professional Experience

### Principal Architect - Web and Smart TV Applications
**Paramount Streaming** | 2020 - Present
- Led architecture for Paramount+ Web during global rollout and code merge
- Architected infrastructure for Super Bowl 58 streaming on Paramount+
- Key digital leader for Paramount+ platform
- Managed web and Smart TV application architecture

### Technical Lead
**Anthem (now Elevance Health)** | 2018 - 2020
- Led technical integration of Tele-Health features into Sydney Health platform
- Integrated AI capabilities into healthcare applications
- Delivered projects with global impact

## Skills
- **Languages**: JavaScript, TypeScript, Python
- **Frameworks**: React, Node.js
- **Platforms**: Web, Smart TV (Roku, Fire TV, Samsung, LG)
- **Cloud**: AWS, Azure
- **Technical Leadership**: Multi-National Teams
- **Architecture**: Microservices, Event-driven architecture
- **Streaming**: Video streaming protocols, CDN optimization

## Education
Computer Science and Computer Engineering
Christopher Newport University

- Dislplay the generated `interview_materials.md` file.

In [34]:
display(Markdown("./interview_materials.md"))

Based on Tim Masterson's resume and the job requirements for the AI Fund position, here are some potential interview questions and talking points:

1. Can you provide an example of a large-scale project you have worked on that had a global impact, and how did you ensure its success?
2. How have you utilized your technical leadership skills to manage multi-national teams effectively in previous roles?
3. Could you explain your experience in architecting infrastructure for streaming platforms and web applications, specifically in relation to the technologies you have worked with (JavaScript, TypeScript, Python, React, Node.js)?
4. When integrating AI capabilities into healthcare applications, what challenges did you face, and how did you overcome them?
5. Describe a time when you had to lead the technical integration of new features into an existing platform. What was the outcome, and what strategies did you implement to ensure a successful integration?
6. How have you leveraged your expertise in event-driven architecture and microservices to improve the scalability and performance of applications you have worked on?
7. Can you discuss your experience with video streaming protocols and CDN optimization, and how you have applied this knowledge in your previous roles?
8. How do you stay current with industry trends and advancements in technology, and how do you incorporate this knowledge into your work as a Principal Architect?

These questions and talking points are designed to help the candidate showcase their relevant experience, skills, and accomplishments that align with the requirements of the AI Fund position.